# MLT Task #2
Implemented by Gaibaliev Emil

In [15]:
import urllib.request
import re
import nltk
from nltk.tokenize import TreebankWordTokenizer
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer, WordNetLemmatizer
from collections import Counter
import copy
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')
from sklearn.feature_extraction.text import TfidfVectorizer

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\emil_\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\emil_\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\emil_\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\emil_\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


## Data preprocessing

In [2]:
url = 'https://www.gutenberg.org/files/11/11-0.txt'
response = urllib.request.urlopen(url)
data = response.read()
text = data.decode('utf-8')

In [3]:
chapters = text.split('CHAPTER')

In [4]:
chapters = chapters[13:]

In [5]:
for i in range(len(chapters)):
    chapters[i] = re.sub("<.*?>", ' ', chapters[i])
    chapters[i] = re.sub("\n|\r|_", ' ', chapters[i])
    chapters[i] = re.sub(" +", ' ', chapters[i])
    chapters[i] = chapters[i].lower()
    chapters[i] = nltk.sent_tokenize(chapters[i])

In [6]:
stop_words = stopwords.words("english")
stop_words_alice = stop_words.copy()
stop_words_alice.append('alice')

In [7]:
lemmatizer = WordNetLemmatizer()
stemmer = PorterStemmer()

In [8]:
sentences = copy.deepcopy(chapters)

In [9]:
for i in range(len(chapters)):
    for j in range(len(chapters[i])):
        chapters[i][j] = re.sub(r"[^\w\s]", '', chapters[i][j])
        chapters[i][j] = TreebankWordTokenizer().tokenize(chapters[i][j])
        for k in range(len(chapters[i][j])):
            if chapters[i][j][k] in stop_words_alice:
                chapters[i][j][k] = ''
            chapters[i][j][k] = lemmatizer.lemmatize(chapters[i][j][k])
            #chapters[i][j][k] = stemmer.stem(chapters[i][j][k])
        chapters[i][j] = ' '.join(chapters[i][j])
        chapters[i][j] = re.sub(' +', ' ', chapters[i][j])
        chapters[i][j] = chapters[i][j].strip()
    chapters[i] = list(filter(None, chapters[i]))

In [10]:
for i in range(len(sentences)):
    for j in range(len(sentences[i])):
        sentences[i][j] = re.sub(r"[^\w\s]", '', sentences[i][j])
        sentences[i][j] = TreebankWordTokenizer().tokenize(sentences[i][j])
        for k in range(len(sentences[i][j])):
            if sentences[i][j][k] in stop_words:
                sentences[i][j][k] = ''
            sentences[i][j][k] = lemmatizer.lemmatize(sentences[i][j][k])
            #chapters[i][j][k] = stemmer.stem(chapters[i][j][k])
        sentences[i][j] = ' '.join(sentences[i][j])
        sentences[i][j] = re.sub(' +', ' ', sentences[i][j])
        sentences[i][j] = sentences[i][j].strip()
    sentences[i] = list(filter(None, sentences[i]))

In [11]:
for i in range(len(chapters)):
    chapters[i] = ' '.join(chapters[i])

## Top 10 the most important words

In [12]:
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(chapters)

# Получим массив слов
feature_names = vectorizer.get_feature_names_out()

In [13]:
for i, chapter in enumerate(X):
    # Получим значения TF-IDF для данной главы
    tfidf_scores = zip(feature_names, chapter.toarray()[0])
    # Отсортируем по убыванию TF-IDF
    sorted_items = sorted(tfidf_scores, key=lambda x: x[1], reverse=True)
    # Выберем топ-10 слов
    top10 = sorted_items[:10]
    print(f"Глава {i+1}: {top10}")

Глава 1: [('little', 0.17220521068853128), ('bat', 0.16988643782420274), ('door', 0.15348731183768863), ('key', 0.15007029706505384), ('eat', 0.142497851203876), ('like', 0.12628382117158962), ('think', 0.12628382117158962), ('way', 0.12628382117158962), ('either', 0.122141015317608), ('see', 0.1148034737923542)]
Глава 2: [('mouse', 0.3039552756661129), ('pool', 0.18672948802288214), ('little', 0.18213079295020249), ('im', 0.16254319506595163), ('swam', 0.15383635519858957), ('cat', 0.15197763783305646), ('dear', 0.14876879113377037), ('said', 0.12856291267073117), ('foot', 0.12503322697380898), ('mabel', 0.12306908415887165)]
Глава 3: [('mouse', 0.39836073809127703), ('said', 0.3637314924975534), ('dodo', 0.3166180457768027), ('prize', 0.18433498093449507), ('lory', 0.15830902288840135), ('dry', 0.13984331178645862), ('thimble', 0.12288998728966337), ('know', 0.11767783580803198), ('bird', 0.11381735374036486), ('soon', 0.09453327840977521)]
Глава 4: [('bill', 0.21328127215206638), ('

 CHAPTER I.     Down the Rabbit-Hole              - A House of Little Bat.</br>
 CHAPTER II.    The Pool of Tears                 - Cat, Mouse, and Pool.</br>
 CHAPTER III.   A Caucus-Race and a Long Tale     - The Competition.</br>
 CHAPTER IV.    The Rabbit Sends in a Little Bill - The Rabbit with a Little Bill.</br>
 CHAPTER V.     Advice from a Caterpillar         - Advice from a Caterpillar.</br>
 CHAPTER VI.    Pig and Pepper                    - Cat and Footman.</br>
 CHAPTER VII.   A Mad Tea-Party                   - A Tea-Party.</br>
 CHAPTER VIII.  The Queen’s Croquet-Ground        - Reception at the Queen's.</br>
 CHAPTER IX.    The Mock Turtle’s Story           - The Mock Turtle's Story.</br>
 CHAPTER X.     The Lobster Quadrille             - The Party.</br>
 CHAPTER XI.    Who Stole the Tarts?              - The Court.</br>
 CHAPTER XII.   Alice’s Evidence                  - Royal Favor.</br>

## The top 10 the most used verbs

In [22]:
def find_top_verbs_with_alice(sentences):
    lemmatizer = WordNetLemmatizer()
    verb_counts = Counter()

    for chapter in sentences:
        for sentence in chapter:
            if "alice" in sentence:
                words = nltk.word_tokenize(sentence)
                for word in words:
                    lemma = lemmatizer.lemmatize(word, pos='v')
                    if lemma != 'alice' and lemma.isalpha():
                        verb_counts[lemma] += 1

    return verb_counts.most_common(10)

top_verbs = find_top_verbs_with_alice(sentences)
print(top_verbs)

[('say', 298), ('think', 98), ('go', 93), ('get', 65), ('little', 60), ('look', 58), ('like', 58), ('know', 57), ('come', 49), ('begin', 48)]
